In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
#         print("when all fails, remember you didn't flatten the input. Bitch")
        self.block_length = 10;
        self.state_vector_length = 3; # state_vector = (Px, Py, Pz)
        self.control_input_length = 3; # control_input = (Fx, Fy, Fz)
        self.visible_features = self.block_length * (self.state_vector_length + self.control_input_length)
        self.output_hidden = 100; # Noh
        self.latent_features = 50; # Nl
        self.tru_units = 50; # Nlh
        '''
        TRU unit : TRU = {h_lp, h_lc} and l_b_prev
        * assuming flattened-out input vectors of size (feature*time_block_length)
        
        '''
        self.h_lp = nn.Linear(self.visible_features, self.output_hidden, bias = False);
        self.h_lc = nn.Linear(self.visible_features, self.output_hidden, bias = False);
        self.l_b_prev = torch.rand(self.output_hidden);
        
        '''
        Rest of network aka Conditional Dynamic Responses section 
        CDR = {h_l, h_c, h_f}
        '''
        self.h_c =  nn.Linear(self.visible_features, self.output_hidden, bias = False);
        self.h_f = nn.Linear(self.control_input_length * self.block_length, self.output_hidden, bias = False);
        self.h_l = nn.Linear(self.latent_features, self.output_hidden, bias = False);

    def forward(self, x):
        x = x.view(-1, self.num_flat_features(x))
        '''
        So, since pytorch is in truth a little bitch, Imma go on and assume that
        the input variable x is a tuple of (v_b, u_b_next) <- and now I am confused cause
        u_b_next is supposed to be the output of  the MPC and that thingy is NOT trained
        only online. So, new idea, let's try to isolate the previous parts of the network, 
        in which case the input is v_b
        '''
        s = F.Sigmoid()
        
        '''
        TRU pass 
        h_lc : Wc * v_b
        h_lp : Wf * v_b_prev
        '''
        x = s(self.h_lc(x));
        temp = s(self.h_lp(self.l_b_prev));
        x = x * temp;
        x = x + self.l_b_prev;
        x = s(x);
        self.l_b_prev = x;
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
print(list(net.parameters()))

Net(
  (h_lp): Linear(in_features=60, out_features=100, bias=False)
  (h_lc): Linear(in_features=60, out_features=100, bias=False)
  (h_c): Linear(in_features=60, out_features=100, bias=False)
  (h_f): Linear(in_features=30, out_features=100, bias=False)
  (h_l): Linear(in_features=50, out_features=100, bias=False)
)
[Parameter containing:
tensor([[-0.1144, -0.0913,  0.0134,  ..., -0.0179,  0.0366, -0.0800],
        [ 0.0310,  0.1268,  0.0688,  ...,  0.1161, -0.0535, -0.0967],
        [-0.0192, -0.1107,  0.0344,  ...,  0.0892, -0.0781,  0.1085],
        ...,
        [-0.0049, -0.0920, -0.0387,  ...,  0.0410, -0.0537, -0.0271],
        [ 0.0057,  0.0516,  0.0181,  ...,  0.0810,  0.0922,  0.0146],
        [ 0.0453,  0.0929, -0.0525,  ..., -0.0208,  0.0633,  0.0354]],
       requires_grad=True), Parameter containing:
tensor([[ 0.0561, -0.0538,  0.1261,  ..., -0.0370, -0.0854,  0.0028],
        [-0.0828,  0.0712, -0.1020,  ..., -0.1130,  0.0337, -0.0797],
        [-0.1250,  0.0315,  0.0308